In [4]:
import pulp

# Define the problem
problem = pulp.LpProblem("Time Tabling and Vehicle Scheduling", pulp.LpMinimize)

# Define sets as lists or sets in Python

# N: Set of Terminals (e.g., bus stops excluding the depot)
N = {'Node1', 'Node2333', 'Node3'}

# N+ : Set of nodes including N and the depot
N_plus = N.union({'Depot'})

# D: Set of Directions
D = {'inbound', 'outBound'}

# T: Set of Trips
T = {'Trip1', 'Trip 2', 'Trip3' }

Td = {d: [] for d in D}  # Dictionary to store trips for each direction

# Tini_d: Subset of Td comprising initial trips for each Direction d in D
Tini_d = {d: {'Trip1'} for d in D}  # Example initial trip for each direction

# Tfin_d: Subset of Td comprising final trips for each Direction d in D
Tfin_d = {d: {'Trip3'} for d in D}  # Example final trip for each direction

# Tstd_d: Intermediate trips for each Direction d
Tstd_d = {d: set(Td[d]) - Tini_d[d] - Tfin_d[d] for d in D}

# Tini and Tfin: Unions of initial and final trips across all directions
Tini = set().union(*Tini_d.values())
Tfin = set().union(*Tfin_d.values())

# H: Set of Time Windows as tuples (a, b)
H = {(8, 10), (10, 12), (12, 14)}  # Example time windows

# V: Set of Vehicles
V = {'Combustion1', 'Combustion2'}



In [5]:
# Define parameters as dictionaries based on sets and values

# a: Arrival Time for each Trip t in T
a = {
    'Trip1': 8,  # example times in hours
    'Trip2': 9,
    'Trip3': 10
}

# h: Time Window for each trip t in T
h = {
    'Trip1': (8, 10),
    'Trip2': (10, 12),
    'Trip3': (12, 14)
}

# sn: Start Terminal for each Trip t in T
sn = {
    'Trip1': 'Stop1',
    'Trip2': 'Stop2',
    'Trip3': 'Stop3'
}

# en: End Terminal for each Trip t in T
en = {
    'Trip1': 'Stop2',
    'Trip2': 'Stop3',
    'Trip3': 'Stop1'
}

# st: Start Time at the initial stop for each Trip t in T
st = {
    'Trip1': 8,
    'Trip2': 9,
    'Trip3': 10
}

# et: End Time at the final stop for each Trip t in T
et = {
    'Trip1': 9,
    'Trip2': 10,
    'Trip3': 11
}

# Ih_d_max: Maximum Headway for each direction d in D and each time window h in H
Ih_d_max = {
    ('Direction1', (8, 10)): 15,  # example headway in minutes
    ('Direction1', (10, 12)): 20,
    ('Direction2', (12, 14)): 15
}

# δh(i)_en(i),min: Minimum stopping time at the end terminal of each trip i in T
delta_min_end_terminal = {
    'Trip1': 5,  # example minimum stopping times in minutes
    'Trip2': 6,
    'Trip3': 7
}

# δh(i)_en(i),max: Maximum stopping time at the end terminal of each trip i in T
delta_max_end_terminal = {
    'Trip1': 10,  # example maximum stopping times in minutes
    'Trip2': 12,
    'Trip3': 15
}

# δh_depot,min: Minimum stopping time at the depot in each time window h in H
delta_min_depot = {
    (8, 10): 10,
    (10, 12): 15,
    (12, 14): 12
}

# δh_depot,max: Maximum stopping time at the depot in each time window h in H
delta_max_depot = {
    (8, 10): 20,
    (10, 12): 25,
    (12, 14): 20
}

# th_n+, t pull-out travel time from terminal n to depot in each window h in H
t_pull_out = {
    ('Stop1', (8, 10)): 15,
    ('Stop2', (10, 12)): 10,
    ('Stop3', (12, 14)): 12
}

# th_n-, t pull-in travel time from depot to terminal n in each window h in H
t_pull_in = {
    ('Stop1', (8, 10)): 10,
    ('Stop2', (10, 12)): 15,
    ('Stop3', (12, 14)): 12
}

M = len(Td)


In [6]:
# Variables
x = pulp.LpVariable.dicts(f"Do we do the trip t?",
                          T,
                          cat="Binary")
Z_t_v = pulp.LpVariable.dicts(f"Vehicle_Assignment",
                              ((t, v) for t in T for v in V),
                              cat="Binary") # The vehicle v drives on the trip t


In [7]:
#Nebenbedingungen

# TT-Feasibility Constraints

# Constraint (1): Modeling the Timetabling
for d in D:
    problem += pulp.lpSum(x[t] for t in Tini_d[d]) == 1

# Constraint (2): Final trips are realized for each direction d in D
for d in D:
    problem += pulp.lpSum(x[t] for t in Tfin_d[d]) == 1

# Constraint (3): Headway constraint
for d in D:
    for i in Tstd_d[d]:
        problem += -x[i] + pulp.lpSum(x[j] for j in Tstd_d[d]) >= -M * (1 - x[i]) # if trip i is not scheduled (x[i] = 0), the constraint does not affect the solution. If trip i is scheduled (x[i] = 1), -1 + pulp.lpSum(x[j] for j in Tstd_d[d]) >= -M * 0 and This ensures that at least one trip in the set of intermediate trips is scheduled.
